In [14]:
import glob
import numpy as np
import cv2



# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.\
objpoints = [] # 3d point in real world spaceq
imgpoints = [] # 2d points in image plane.

camera = cv2.VideoCapture(0, cv2.CAP_DSHOW)

import time

while True:
    successful, img = camera.read()
    if successful:
        img = cv2.flip(img,1)

        cv2.imshow("Webcam", img) # This will open an independent window
            

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (6,7),None)
        # If found, add object points, image points (after refining them)
        if ret == True:
            time.sleep()
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)
            # Draw and display the corners
            cv2.drawChessboardCorners(img, (7,6), corners2, ret)
            cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cv2.destroyAllWindows()
        camera.release()
        break
    


KeyboardInterrupt: 

Calibration calculation     

In [ ]:


img = gray
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
cv2.imshow('Un-calibrated image',img)

h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

cv2.imshow('Calibrated image', dst)


while True:
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cv2.destroyAllWindows()
        camera.release()
        break
    
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )



total error: 14.925081755189709


Pose Estimator  

In [16]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

camera= cv2.VideoCapture(1)
while True:
    
    success, img = camera.read()
    if success:        
        img = cv2.flip(img,1)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (7,6),None)
        if ret == True:
            corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            # Find the rotation and translation vectors.
            ret,rvecs, tvecs = cv2.solvePnP(objp, corners2, mtx, dist)
            # project 3D points to image plane
            imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
            img = draw(img,corners2,imgpts)
            cv2.imshow('img',img)
       
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        camera.release()
        out.release()
        break
cv2.destroyAllWindows()

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type


save to disk

In [ ]:
import cv2 as cv2

camera = cv2.VideoCapture(1,cv2.CAP_DSHOW)

#camera = cv2.VideoCapture(0)

frame_width = int(camera.get(3))
frame_height = int(camera.get(4))
camera.set(3,frame_width) # adjust width
camera.set(4,frame_height) # adjust height
size = (frame_width, frame_height)



fourcc = cv2.VideoWriter_fourcc(*'XVID')

out = cv2.VideoWriter('opencv-video.mp4',cv2.VideoWriter_fourcc(*'H264'),30, size)


while True:
    
    success, img = camera.read()
    if success:
        cv2.imshow("Webcam", img) # This will open an independent window
        out.write(img)
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        camera.release()
        out.release()
        break
cv2.destroyAllWindows()

<class 'int'>


In [ ]:
print("Saving")


ret_values = []

mtx_values = []
dist_values = []
rvecs_values = []
tvecs_values = []

# Rest of the code remains the same

for fname in glob.glob('left*.jpg'):
    # Rest of the code remains the same

    if ret == True:
        # Rest of the code remains the same

        # Save the variables
        ret_values.append(ret)
        mtx_values.append(mtx)
        dist_values.append(dist)
        rvecs_values.append(rvecs)
        tvecs_values.append(tvecs)

# Save the variables to a .npz file
np.savez('calibration.npz', ret=ret_values, mtx=mtx_values, dist=dist_values, rvecs=rvecs_values, tvecs=tvecs_values)


Saving
